[색인 & 가중치 => 학습]
1. 수집한 문서 목록 가져오기
2. 각 문서에서 색인어 목록 추출 전처리 -> 어절 -> 형태소+명사+바이그램
3. 가중치 계산(TF-IDF) -> TDM -> TWM(with DocumentVectorLength)

[질의 -> 색인과정의 2 to 3]
4. 질의에서 색인어 추출
5. 가중치 계산
6. 유사도 계산(코싸인)
7. 유사도 순 정렬(거리-오름차순, 각도-내림차순)
8. 검색 결과 출력


## 1. 모든 문서들에 대해 토큰화 수행하기

In [64]:
# 이 코드에서는 지금까지 배운 내용들을 활용해 텍스트 문서에서 쿼리 단어를 검색한 후 해당 쿼리가 위치한 문서 경로를 출력해 봅니다. 

# 파이썬 버전확인
import sys
print("--sys.version--")
print(sys.version)

print("\n--sys.version_info--")
print(sys.version_info)

print("\n--sys.hexversion--")
print(sys.hexversion)


def getFileList(base = "./", ext = "txt"): # 아무것도 안했다면 base는 현재 경로
    fileList = list()
    for file in listdir(base):
        if file.split(".")[-1] == ext: # .을 기준으로 split한 것이 txt인지 검사
            fileList.append("{0}/{1}".format(base, file))
            
    return fileList

# txt 컨텐츠를 편하게 읽어오기 위한 함수
def getContent(file): 
    with open(file, encoding="UTF-8") as f:
        content = f.read()
    return content

--sys.version--
3.7.1 (default, Dec 10 2018, 22:54:23) [MSC v.1915 64 bit (AMD64)]

--sys.version_info--
sys.version_info(major=3, minor=7, micro=1, releaselevel='final', serial=0)

--sys.hexversion--
50790896


In [65]:
def ngramEojeol(sentence, n=2): # sentence를 받아 어절 단위로 분리해주는 함수
    '''
    입력:     단어1,   단어2,   단어3,  단어4 : 4
    출력(2) : 단어12,  단어23,  단어34 :        3 - n + 1
    출력(3) : 단어123, 단어234         :        2 - n + 1
    '''
    tokens = sentence.split()
    ngram = []
    
    for i in range(len(tokens) - n + 1):
        ngram.append(' '.join(tokens[i:i + n]))    
        
    return ngram

In [66]:
def ngramUmjeol(term, n = 2): # 음절 단위로 구분하는 함수. sentence를 받아 2개(n=2)씩 쪼갠다.

    ngram = []
    
    for i in range(len(term) - n + 1):
        # ngram.append(tokens_ngram[i:i+n]) # 방법1
        # ngram.append(tuple(tokens_ngram[i:i+n])) # 방법2 (튜플로 반환 시 키값을 쓸 수 있음)
        ngram.append(''.join(term[i:i + n])) # 방법3
        
    return ngram

In [67]:
import re
from string import punctuation


def getPatternList():
    patternList = dict()
    pattern = re.compile(r"[%s]{1,}" % re.escape(punctuation)) # punctuation 안의 특수문자가 두번이상 반복되는 모든 문자에 대해 패턴 정의
    patternList["Punctuation"] = pattern
    pattern2 = re.compile(r"([A-Za-z0-9\-\_\.]{3,}@[A-Za-z0-9\-\_]{3,}(.[A-Za-z]{2,})+)") #이메일주소제거패턴
    patternList["Email"] = pattern2
    pattern3 = re.compile(r"([A-Za-z0-9\-\_]{1,}(.[A-Za-z]{2,})+)") # 신문사도메인패턴
    patternList["Domain"] = pattern3
    pattern4 = re.compile(r"\s{2,}") # 두칸짜리 공백제거
    patternList["Whitespace"] = pattern4
    pattern5 = re.compile(r"([^ㄱ-ㅎㅏ-ㅣ가-힣0-9]+)") # 한글이 아닌 영어 기호 제거
    patternList["Korean"] = pattern5
    pattern6 = re.compile(r'[A-Za-z-_]{8,}') # 영어 (대소문자) +\-\_ 제거 (8글자 이상)
    patternList["ElimLongEng"] = pattern6
    pattern7 = re.compile(r"를 2019 및 금지|네이버 채널에서|저작권자 주 블로터앤미디어 저작권자를 명기하고 내용을 변경하지 않으며 비상업적으로 이용하는 조건아래 가능합니다|"
                          r"디지털 마케팅의 미래 아이포럼|무단전재|오류를 우회하기 위한 함수 추가|재배포|기자|아이뉴스24|구독해주세요|"
                          r"고수들의 재테크 비법|이건희칼럼|성공을 꿈꾸는 사람들의 경제 뉴스|머니 및 금지|및 금지|"
                          r"네이버 모바일에서|네이버에서 헤럴드경제|채널 구독하기|헤럴드 리얼라이프 헤럴드경제 사이트 바로가기 헤럴드경제 무단 전재 및 금지|"
                          r"전자신문|바로가기|전자신문인터넷|인터넷 및 금지|중앙일보|친구추가|구독 1위|"
                          r"한국경제|재배포|기사제보|보도자료|한경닷컴|네이버에서|자세히보기|자세히 보기|모바일한경|구독신청|"
                          r"머니투데이|돈이 보이는 리얼타임 뉴스|연예뉴스|파이낸셜뉴스|무단 전재 금지|"
                          r"C|Users|brsta|brstar|txt") # 높은 가중치를 갖는 단어를 하드코딩 데이터로 제거
    patternList["ElimRecWord"] = pattern7
    pattern8 = re.compile(r'[0-9]') # 숫자만 제거
    patternList["ElimNumOnly"] = pattern8
    pattern9 = re.compile(r"\s{1,}") # 한칸짜리 공백제거
    patternList["Whitespace_one"] = pattern9
    
    return patternList

In [68]:
from os import listdir

# FileList = getFileList("C:/Users/brsta/ICT_AI_AdvanceClass_NLP/0314_DownloadedNewstxts/")
FileList = getFileList("./0314_DownloadedNewstxts")

NewsContent = list() 

for i in range(len(FileList)):
    NewsContent.append(getContent(FileList[i])) # len(NewsContent) = 180


In [69]:
patternList = getPatternList()
for _ in ["Korean", "Whitespace", "Punctuation", "ElimLongEng", "Email", "Domain", "ElimRecWord"]: 
    for i in range(len(FileList)):
        NewsContent[i] = patternList[_].sub(" ", NewsContent[i]) 
print(NewsContent[0]) # 불용어 처리가 모두 끝난 뉴스기사 목록


   오히려 단체측 무선이어폰 연구는 존재치도 않아 국내 참여 연구자도 어리둥절 이엠에프사이언티스에서 일부 언론 보도를 부정하며 직접 보내온 이메일 내용이다 이엠에프사이언티스트 관계자인 조엘 모스코위츠 미국 버클리 캘리포니아대 가정사회건강연구소 소장은 무선 이어폰의 건강 유해성에 대한 보도를 부정했다 고재원   1212 18일 오전 전세계 과학자들이 애플 에어팟과 같은 무선 이어폰이 암 발생 위험을 키울 수 있다는 호소문을 국제연합 과 세계보건기구 에 제출했다는 일부 국내외 언론의 보도가 나오면서 불안감이 확산되고 있다 하지만 실제 이 단체와 호소문에 이름을 올린 과학자들에게 확인한 결과 호소문은 4년전 제출됐던 것이며 또 특정 제품이나 제조사를 언급하지 않은 것으로 확인됐다 18일 데일리메일과   등 국내외 일부 언론에 따르면 전 세계 과학자 247명이 무선 이어폰의 비이온화 전자기장 이 암을 유발할 위험 우려가 있다며 과 에 호소문을 제출했다고 전했다 이들 매체들은 호소문에 애플 에어팟이 에 관한 법적 기준치를 준수하고 있지만 장시간 노출될 경우 건강에 좋지 않을 수 있다 는 내용이 포함됐다고 보도했다 이들 매체들은 이 호소문에는 전 세계 42개국 과학자 247명이 서명을 했다고 전했다 여기에는 한국의 연세대 한양대 가톨릭대 단국대 중앙대 경북대 한림대 소속 과학자 15명의 이름도 포함됐다 하지만 취재 결과 호소문 작성을 주도한 비영리단체 이엠에프사이언티스트 는 애플 에어팟과 같은 무선 이어폰에 대한 유해성을 주장하지 않은 것으로 확인됐다 동아사이언스가 이엠에프사이언티스트에 직접 이메일로 확인한 결과 일부 언론 보도가 호소문에 대한 부정확한 내용을 담고 있다 며 무선 블루투스의 자기장에 머리가 장시간 노출될 시의 안정성에 대한 연구는 존재하지 않는다 고 밝혔다 이엠에프사이언티스트에 따르면 이 단체는 지난 2015년 5월 실제로 전세계 과학자 190명의 서명을 받아 과 유엔환경계획 에 국제 과학자 호소문 을 제출하기는 했다 당시 호소문에는 전세계 저명한 학술

## 2. trainset과 validationset 분리

In [219]:
# train_test split 수행
from sklearn.model_selection import train_test_split
X_train, X_validation = train_test_split(collection, test_size=0.2, train_size=0.8, random_state=410, shuffle=False)
# X_test는 추후 제공

In [220]:
print(len(X_train)), print(len(X_validation))

144
36


(None, None)

In [221]:
X_train

[('./0314_DownloadedNewstxts/IT과학-0000003610.txt',
  '   오히려 단체측 무선이어폰 연구는 존재치도 않아 국내 참여 연구자도 어리둥절 이엠에프사이언티스에서 일부 언론 보도를 부정하며 직접 보내온 이메일 내용이다 이엠에프사이언티스트 관계자인 조엘 모스코위츠 미국 버클리 캘리포니아대 가정사회건강연구소 소장은 무선 이어폰의 건강 유해성에 대한 보도를 부정했다 고재원   1212 18일 오전 전세계 과학자들이 애플 에어팟과 같은 무선 이어폰이 암 발생 위험을 키울 수 있다는 호소문을 국제연합 과 세계보건기구 에 제출했다는 일부 국내외 언론의 보도가 나오면서 불안감이 확산되고 있다 하지만 실제 이 단체와 호소문에 이름을 올린 과학자들에게 확인한 결과 호소문은 4년전 제출됐던 것이며 또 특정 제품이나 제조사를 언급하지 않은 것으로 확인됐다 18일 데일리메일과   등 국내외 일부 언론에 따르면 전 세계 과학자 247명이 무선 이어폰의 비이온화 전자기장 이 암을 유발할 위험 우려가 있다며 과 에 호소문을 제출했다고 전했다 이들 매체들은 호소문에 애플 에어팟이 에 관한 법적 기준치를 준수하고 있지만 장시간 노출될 경우 건강에 좋지 않을 수 있다 는 내용이 포함됐다고 보도했다 이들 매체들은 이 호소문에는 전 세계 42개국 과학자 247명이 서명을 했다고 전했다 여기에는 한국의 연세대 한양대 가톨릭대 단국대 중앙대 경북대 한림대 소속 과학자 15명의 이름도 포함됐다 하지만 취재 결과 호소문 작성을 주도한 비영리단체 이엠에프사이언티스트 는 애플 에어팟과 같은 무선 이어폰에 대한 유해성을 주장하지 않은 것으로 확인됐다 동아사이언스가 이엠에프사이언티스트에 직접 이메일로 확인한 결과 일부 언론 보도가 호소문에 대한 부정확한 내용을 담고 있다 며 무선 블루투스의 자기장에 머리가 장시간 노출될 시의 안정성에 대한 연구는 존재하지 않는다 고 밝혔다 이엠에프사이언티스트에 따르면 이 단체는 지난 2015년 5월 실제로 전세계 과학자 190명의 서명을 받

## 3. TF-IDF 방식과 cosine simmilarity으로 검색해보기

In [222]:
from math import log10

# TF 구하기
def binaryTF(freq):
    if freq > 0:
        return 1
    else:
        return 0

def rawTF(freq):
    return freq

def basicTF(freq, totalFreq):
    return freq/totalFreq

def logTF(freq):
    if freq > 0:
        return 1+log10(freq)
    else:
        return 0

def doubleNormalTF(K, freq, maxFreq): 
    return K + ((1-K) * (freq/maxFreq))

In [223]:
# IDF(Inverse Document Frequency 구하기
def unaryIDF():
    return 1

def basicIDF(N, df):
    return log10(N/df)

def smoothigIDF(N, df):
    return log10((N+1)/df)

def probabilityIDF(N, df):
    return log10((N-df+1)/df)

In [224]:
def innerProduct(x, y): # 내적 연산
    return x * y

In [359]:
from nltk.tokenize import sent_tokenize, word_tokenize
from konlpy.tag import Kkma
ma = Kkma()

# 단일 뉴스 기사를 토큰화해주는 함수
def Tokenizer(NewsContent):
    dictTerm = list()
    dictPos = list()
    dictNoun = list()
    dictNgram = list()
    th = 1 
    
    for sentence in sent_tokenize(NewsContent):
        for token in word_tokenize(sentence):
            if len(token) > th:
                dictTerm.append(token)
                # 아래서부터는 list이기때문에 extend를 사용해야 함. 
                dictPos.extend([morpheme for morpheme in ma.morphs(token) if len(morpheme) > th]) # 형태소 분석결과를 extend
                dictNoun.extend([noun for noun in ma.nouns(token) if len(noun) > th]) # 명사 단위로 잘라 extend
                dictNgram.extend(ngramUmjeol(token)) # 바이그램을 리턴
                
    # 빠른 속도 및 중복 데이터 처리를 위해 set사용 (유일한 단어만 남김)
    dictTerm = list(set(dictTerm))
    dictPos = list(set(dictPos))
    dictNoun = list(set(dictNoun))
    dictNgram = list(set(dictNgram))
        
    return set(dictPos + dictNgram)

In [273]:
import sys

# Progress Bar함수
def printProgress (iteration, total, prefix = '', suffix = '', decimals = 1, barLength = 100):
    formatStr = "{0:." + str(decimals) + "f}"
    percent = formatStr.format(100 * (iteration / float(total)))
    filledLength = int(round(barLength * iteration / float(total)))
    bar = '#' * filledLength + '-' * (barLength - filledLength)
    sys.stdout.write('\r%s |%s| %s%s %s' % (prefix, bar, percent, '%', suffix)),
    if iteration == total:
        sys.stdout.write('\n')
    sys.stdout.flush()
    
# for i in range(0, 100):
#   printProgress(i, 100, 'Progress:', 'Complete', 1, 50)

In [227]:
import nltk
# nltk.download('punkt') # 최초 1회만 실행

globalLexicon = dict()
globalDocument = list()
globalPosting = list()
iterationCount = 0
entireDocCount = len(X_train)

for (docName, docContent) in X_train:
    docIdx = len(globalDocument)
    globalDocument.append(docName)
    
    localPosting = dict()
    
    for term in Tokenizer(docContent):
        if term not in localPosting.keys():
            localPosting[term] = 1
        else:
            localPosting[term] += 1
    
    maxFreq = max(localPosting.values())
    
    for indexTerm, termFreq in localPosting.items():
        if indexTerm not in globalLexicon.keys():
            lexiconIdx = len(globalLexicon)
            postingIdx = len(globalPosting)
            postingData = [lexiconIdx, docIdx, doubleNormalTF(0, termFreq, maxFreq), -1]
            globalPosting.append(postingData)
            globalLexicon[indexTerm] = postingIdx
        else:
            lexiconIdx = list(globalLexicon.keys()).index(indexTerm)
            postingIdx = len(globalPosting)
            beforeIdx = globalLexicon[indexTerm]
            postingData = [lexiconIdx, docIdx, doubleNormalTF(0, termFreq, maxFreq), beforeIdx]
            globalPosting.append(postingData)
            globalLexicon[indexTerm] = postingIdx
    iterationCount += 1
    printProgress(iterationCount, entireDocCount,'Progress:', 'Complete', 1, 100)

Progress: |####################################################################################################| 100.0% Complete


In [228]:
globalPosting

[[0, 0, 1.0, -1],
 [1, 0, 1.0, -1],
 [2, 0, 1.0, -1],
 [3, 0, 1.0, -1],
 [4, 0, 1.0, -1],
 [5, 0, 1.0, -1],
 [6, 0, 1.0, -1],
 [7, 0, 1.0, -1],
 [8, 0, 1.0, -1],
 [9, 0, 1.0, -1],
 [10, 0, 1.0, -1],
 [11, 0, 1.0, -1],
 [12, 0, 1.0, -1],
 [13, 0, 1.0, -1],
 [14, 0, 1.0, -1],
 [15, 0, 1.0, -1],
 [16, 0, 1.0, -1],
 [17, 0, 1.0, -1],
 [18, 0, 1.0, -1],
 [19, 0, 1.0, -1],
 [20, 0, 1.0, -1],
 [21, 0, 1.0, -1],
 [22, 0, 1.0, -1],
 [23, 0, 1.0, -1],
 [24, 0, 1.0, -1],
 [25, 0, 1.0, -1],
 [26, 0, 1.0, -1],
 [27, 0, 1.0, -1],
 [28, 0, 1.0, -1],
 [29, 0, 1.0, -1],
 [30, 0, 1.0, -1],
 [31, 0, 1.0, -1],
 [32, 0, 1.0, -1],
 [33, 0, 1.0, -1],
 [34, 0, 1.0, -1],
 [35, 0, 1.0, -1],
 [36, 0, 1.0, -1],
 [37, 0, 1.0, -1],
 [38, 0, 1.0, -1],
 [39, 0, 1.0, -1],
 [40, 0, 1.0, -1],
 [41, 0, 1.0, -1],
 [42, 0, 1.0, -1],
 [43, 0, 1.0, -1],
 [44, 0, 1.0, -1],
 [45, 0, 1.0, -1],
 [46, 0, 1.0, -1],
 [47, 0, 1.0, -1],
 [48, 0, 1.0, -1],
 [49, 0, 1.0, -1],
 [50, 0, 1.0, -1],
 [51, 0, 1.0, -1],
 [52, 0, 1.0, -1],
 [5

In [229]:
N = len(globalDocument)
globalLexiconIDF = dict()
globalDocumentLength = dict()

for indexTerm, postingIdx in globalLexicon.items():
    # indexTerm : 단어, postingIdx :위치(포인터역할), ...
    df = 0
    oldPostingIdx = postingIdx
    
    while True: # ptr Next: -1 
        if postingIdx == -1:
            break
        
        df += 1
        postingData = globalPosting[postingIdx]
        postingIdx = postingData[3]
        
    idf = smoothigIDF(N, df)
    globalLexiconIDF[indexTerm] = idf # 나중에 단어가 나왔을때 idf를 불러올 수 있도록 하기 위함
    postingIdx = oldPostingIdx
    # print('단어 : {0} / IDF : {1}'.format(indexTerm, idf))
    
    while True:
        if postingIdx == -1:
            break
        
        postingData = globalPosting[postingIdx]
        TF = postingData[2]
        globalPosting[postingIdx][2] *= idf
        # print('    Documents:{0} / TF:{1} / TF-IDF:{2}'.format(globalDocument[postingData[1]], 
        #                                                        TF, globalPosting[postingIdx][2]))
        
        postingIdx = postingData[3]
        
        # 색인하는 과정에서 사실상 검색까지 모두 한 셈이 되므로 색인 과정에만 시간이 들어 굉장히 빠른 연산속도를 보임. 
        # 키 에러 방지용(키값이 있다면 제곱해서 누적해주면 됨)
        if postingData[1] not in globalDocumentLength.keys():
            globalDocumentLength[postingData[1]] = globalPosting[postingIdx][2] ** 2
        else:
            globalDocumentLength[postingData[1]] += globalPosting[postingIdx][2] ** 2

In [230]:
query = '이어폰 과 스마트폰' # Document

queryPosting = dict()

for term in Tokenizer(query):
    if term not in queryPosting.keys():
        queryPosting[term] = 1
    else:
        queryPosting[term] += 1
        
maxFreq = max(queryPosting.values())

for indexTerm, termFreq in queryPosting.items():
    queryPosting[indexTerm] = doubleNormalTF(0.5, termFreq, maxFreq)

In [231]:
queryPosting

{'이어': 1.0, '트폰': 1.0, '마트': 1.0, '스마': 1.0, '스마트': 1.0, '이어폰': 1.0, '어폰': 1.0}

In [232]:
candidateList = dict()

for indexTerm, queryWeight in queryPosting.items(): # keys에 있는지 검사 후 query가 있을 경우에 한해 처리하기 위함
    if indexTerm in globalLexicon.keys():
        postingIdx = globalLexicon[indexTerm]
        
        while True: # ptr Next: -1
            if postingIdx == -1:
                break
            
            postingData = globalPosting[postingIdx]
            postingIdx = postingData[3]
            documentWeight = postingData[2]
            
            
            # 키 에러 방지용
            if postingData[1] not in candidateList.keys():
                candidateList[postingData[1]] = innerProduct(queryWeight, documentWeight)
            else:
                candidateList[postingData[1]] += innerProduct(queryWeight, documentWeight)

for documentIdx, sumProduct in candidateList.items():
    candidateList[documentIdx] /= globalDocumentLength[documentIdx]

In [215]:
# 정렬 (reversed = False는 작은값부터 큰 값 순서)
resultList = sorted(candidateList.items(), key=lambda x:x[1], reverse=True)

print('query: ', query)

for i, (documentIdx, distance) in enumerate(resultList):
    print('순위:{0} / 문서:{1} / 유사도:{2}'.format((i+1), globalDocument[documentIdx], distance))
    print('   document:{0}'.format(collection[documentIdx][1]))

query:  이어폰 과 스마트폰
순위:1 / 문서:./0314_DownloadedNewstxts/IT과학-0010681725.txt / 유사도:0.02433789517464926
   document:   세계최초 상용화 기념식은 4월 8일 예정5 단말기 출고가는 140만 원 선으로 협의중5 요금제 5만원 대부터 출발할 듯 인가심사 임박 이데일리 김현아 이재운   대한민국 5 세대 이동통신 상용화 일정이 4월 5일로 확정됐다 이날부터 삼성전자 갤럭시 10 5 로 텔레콤 유플러스에서 5 서비스에 가입할 수 있다 단말기 가격은 140만 원대 5 요금은 최하 데이터 제공 구간이 5만 원대에서 시작될 전망이다 21일 과학기술정보통신부와 업계에 따르면 이날 오후 정부와 업계는 회의를 열고 우리나라 5 상용화 일정을 4월5일로 정했다 다만 정부와 기업이 함께 하는 5 세계최초 기념식은 4월 8일 정도로 예상된다 4월 5일은 미국 1위 통신사인 버라이즌이 밝힌 5 상용화 일정인 4월 11일보다 6일 빠르다 버라이즌은 5 전용폰이 아닌 모토로라 모토 3 5 모토 모드 로 4월 11일 상용화를 발표했는데 우리나라는 삼성의 자체 모바일 엑시노스 9820 와 5 모뎀칩 엑시노스 5100 을 탑재한 5 전용 단말기 갤럭시 10 5 로 상용화한다 버라이즌의 5 폰은 지난해 8월 출시한 모토 3 에 퀄컴 5 모뎀 칩이 장착된 5 모토 모드 를 끼우는 방식이다 따라서 모토 3는 모바일 와 모뎀 칩이 연결안돼 반쪽 5 폰으로 평가받는다 통신사 관계자는 4월 5일 상용화때까지 삼성의 갤럭시 10 5 의 품질 테스트에 만전을 기한다는 계획 이라며 전자의 50 5 는 일단 4월 5일 출시되기는 어렵다 고 말했다 전자 50 5 는 퀄컴의 5 모바일 스냅드래곤 855 와 모바일 5 모뎀칩 50 이 들어가는데 퀄컴은 5월이후를 양산 시점으로 보고 있다 삼성의 갤럭시 10 5 출처 유튜브 5 세계최초 상용화 시기가 4월 5일로 확정됨에 따라 5 요금제도 급물살을 타고 있다 과기정통부는 요금인가권을 

## TDM-DTM-TWM

In [412]:
# DTM 만들기(DocRepresentation)
def getDocReprByDefaultDict(dataset):
    docRepr = defaultdict(lambda: defaultdict(int))
    
    count = 0
    for doc in dataset:
        for PoswithNgram in Tokenizer(doc[1]):
            docRepr[doc[0]][PoswithNgram] += 1
        count +=1
        printProgress(count, entireDocCount,'Progress:', str(count) + 'th Document has been converted to DTM', 1, 50)
    return docRepr

# InvertedDocument만들기
def convertInvertedDocument(DTM):
    TDM = defaultdict(lambda: defaultdict(int))
    entireNum = len(DTM)
    count = 0
    for fileName, termList in DTM.items():  
        maxFreq = max(termList.values())
        for term, freq in termList.items():
            TDM[term][fileName] = doubleNormalTF(0, freq, maxFreq)
        count +=1
        printProgress(count, entireNum,'Progress:', str(count) + 'th Document has been converted to TDM', 1, 50)   
    return TDM # 빈도와 가중치를 동시에 갖게 된다.


# Term - Document => Term-Weight
def TDM2TWM(TDM): # TDM을 받아 term에 대한 weight가 들어있는 값으로 리턴
    N = len(DTM)
    
    TWM = defaultdict(lambda: defaultdict(float))
    DVL = defaultdict(float)
    
    for term, tfList in TDM.items():
        df = len(tfList)
        idf = basicIDF(N, df)
        
        for fileName, tf in tfList.items():
            # 각 문서에서 각 단어가 차지하는 단어
            TWM[term][fileName] = tf * idf
            DVL[fileName] += TWM[term][fileName] ** 2
        
    return TWM, DVL

In [413]:
DTM_train = getDocReprByDefaultDict(X_train)
TDM_train = convertInvertedDocument(DTM_train)

Progress: |##################################################| 100.0% 144th Document has been converted to DTM
Progress: |##################################################| 100.0% 144th Document has been converted to TDM


In [414]:
TWM_train, DVL_train = TDM2TWM(TDM_train)

In [415]:
TWM_train # 0322_main.ipynb와 달리 빈도와 가중치를 동시에 갖게 된다.

defaultdict(<function __main__.TDM2TWM.<locals>.<lambda>()>,
            {'보호': defaultdict(float,
                         {'./0314_DownloadedNewstxts/IT과학-0000003610.txt': 1.255272505103306,
                          './0314_DownloadedNewstxts/IT과학-0000485754.txt': 1.255272505103306,
                          './0314_DownloadedNewstxts/IT과학-0002158229.txt': 1.255272505103306,
                          './0314_DownloadedNewstxts/IT과학-0002794564.txt': 1.255272505103306,
                          './0314_DownloadedNewstxts/경제-0004320497.txt': 1.255272505103306,
                          './0314_DownloadedNewstxts/사회-0010702002.txt': 1.255272505103306,
                          './0314_DownloadedNewstxts/생활문화-0002220787.txt': 1.255272505103306,
                          './0314_DownloadedNewstxts/생활문화-0004107967.txt': 1.255272505103306}),
             '지만': defaultdict(float,
                         {'./0314_DownloadedNewstxts/IT과학-0000003610.txt': 0.14976232033333212,
                 

In [416]:
DVL_train

defaultdict(float,
            {'./0314_DownloadedNewstxts/IT과학-0000003610.txt': 948.2454601790038,
             './0314_DownloadedNewstxts/IT과학-0000485754.txt': 491.7797033308345,
             './0314_DownloadedNewstxts/IT과학-0002158229.txt': 781.3806851447405,
             './0314_DownloadedNewstxts/IT과학-0002794564.txt': 748.1126893496751,
             './0314_DownloadedNewstxts/경제-0004320497.txt': 653.3252462691433,
             './0314_DownloadedNewstxts/사회-0010702002.txt': 961.8718899011922,
             './0314_DownloadedNewstxts/생활문화-0002220787.txt': 2665.13540219051,
             './0314_DownloadedNewstxts/생활문화-0004107967.txt': 1125.7125781596835,
             './0314_DownloadedNewstxts/IT과학-0000023756.txt': 1595.2046388944925,
             './0314_DownloadedNewstxts/IT과학-0000023785.txt': 741.0408211736384,
             './0314_DownloadedNewstxts/IT과학-0000484957.txt': 424.56720320268107,
             './0314_DownloadedNewstxts/IT과학-0001510642.txt': 806.5954246870823,
           

In [417]:
X_validation[0]

('./0314_DownloadedNewstxts/세계-0009120125.txt',
 '   볼턴 빅딜론 겨냥 야심찬 접근 대북외교에 독약 워싱턴 뉴시스 뉴스가 17일 현지시간 트럼프 행정부 대표적 대북 매파 존 볼턴 국가안보보좌관을 해고해야 한다고 주장했다 사진은 볼턴 보좌관이 지난 1월28일 백악관 브리핑룸에서 브리핑을 준비하는 모습 2019 03 18 서울 뉴시스 김난영   하노이 회담 결렬 이후 북미관계 앞날이 불투명한 가운데 트럼프 행정부의 대표적 대북 매파 존 볼턴 국가안보보좌관을 해고해야 한다는 주장이 나왔다 북한 전문매체 뉴스는 17일 현지시간 논평을 통해 이같이 주장했다 뉴스는 하노이 회담 결렬 이후 볼턴 보좌관이 여러 언론과의 인터뷰에서 언급했던 빅딜 론에 대해 그같은 야심찬 접근이 근본적으로 현재 대북 외교 과정에 독약이 된다는 사실을 아는 사람들을 놀라게 했다 고 지적했다 현재 상황에서 볼턴식 빅딜 강조가 북미관계 및 비핵화 협상에 도움이 되지 않는다는 것이다 매체는 또 제2차 북미 정상회담 전 미국과 북한 사이에 오갔던 영변 핵시설 폐기 및 일부 제재 완화 연락사무소 설치 평화협정 등 논의를 거론한 뒤 볼턴 보좌관의 빅딜 발언은 미국의 입장이 훨씬 불합리한 것처럼 보이게 만들었다 고 비판했다 뉴스는 볼턴 보좌관의 대북관에 대해 오랫동안 강경하고 융통성 없는 입장이 혼합돼 있었다 고 혹평했다 이어 하노이 회담 이후 볼턴 보좌관이 미국 대북정책의 공식 얼굴이 됐고 미국은 그 과정에서 외교 과정을 재편할 기회를 낭비했다 고 했다 매체는 볼턴 보좌관의 발언은 워싱턴을 향해 열린 대북 문제 해결의 기회의 창을 단호히 닫았다 며 볼턴 보좌관은 미국의 협상단이 얼마나 유연성있게 해왔는지를 강조하는 대신 미국의 협상 포지션을 불합리하고 지나치게 야심적인 틀에 넣었다 고 했다 뉴스는 볼턴 보좌관의 대북관 및 미국 대북정책 영향에 대해 한결같이 미국이 불합리하고 강경하게 보이도록 하는 데 전력해왔다 고 평가했다 이어 이는 협상 테이블에서 미국의 영향력을

In [418]:
# 문서 하나에 대해서 queryWeight를 반환하는 함수
def QueryWeight(query):
    queryRepr = defaultdict(int)
    N = len(DTM_train)
    
    for MorpwithNgram in Tokenizer(query):
        for morpheme in ma.morphs(MorpwithNgram):
            queryRepr[morpheme] += 1 # 빈도도 쉽게 구할 수 있다. 

    maxFreq = max(queryRepr.values())
    queryWeight = defaultdict(float)
    
    for token, freq in queryRepr.items():
        if token in TWM_train.keys():
            tf = doubleNormalTF(0.5, freq, maxFreq)
            df = len(TWM_train[token])
            idf = basicIDF(N, df)
            queryWeight[token] = tf * idf
    return queryWeight

In [441]:
# 분류를 위해 Query를 주는 부분

print(X_validation[5][1])
queryWeight = QueryWeight(X_validation[0][1])
queryWeight

   베트남 북미 협상 결렬 후 김 위원장 방러 가능성 한층 커져최근 북러 고위인사 접촉 빈번해져 에 러 과의 공조 과시 의도 푸틴 김정은 방러 초청 제작 최자윤 사진합성 일러스트 모스크바 베이징 연합뉴스 유철종 심재훈 특파원 김정은 북한 국무위원장의 최측근 인사인 김창선 북한 국무위원회 부장이 러시아 수도 모스크바를 방문한 것으로 알려졌다 모스크바와 중국 베이징 소식통은 김 위원장의 의전 책임자인 김 부장이 베이징을 거쳐 지난 19일 현지시간 모스크바에 도착했다고 전했다 김 부장은 여전히 모스크바에 머무는 것으로 전해졌다 김 부장은 모스크바에서 러시아 측과 김 위원장의 방러 의전 문제를 논의할 것으로 예상된다 김정은 위원장을 최측근에서 보좌하는 비서실장 격인 김 부장은 김 위원장의 대외 방문 의전 책임자로 그의 모스크바 방문은 김 위원장의 방러가 임박했음을 시사하는 것으로 해석된다 김 부장의 모스크바 방문은 지난달 베트남 북미 협상 결렬 이후 김정은 위원장의 러시아 방문 가능성이 한층 커졌다는 관측이 제기되는 가운데 이루어졌다 북한은 베트남 북미 정상회담에서 자신들이 내세운 요구 조건 수용을 거부한 미국 측을 압박하기 위해 우방인 중국 및 러시아와의 유대 관계를 한층 강화할 것으로 예상돼 왔다 이 같은 맥락에서 김 위원장이 이미 여러 차례 방문한 중국에 이어 조만간 러시아를 찾을 가능성이 커졌다는 분석이 제기됐었다 최근 들어 북러 고위 인사 접촉이 한층 긴밀해진 것도 주목을 받았다 임천일 북한 외무성 부상 차관 이 지난 14일 모스크바를 방문해 이고리 모르굴로프 러시아 외무부 아태지역 담당 차관과 회담했다 김정은 북한 국무위원장 타스 연합뉴스 이에 앞서 김영재 북한 대외경제상도 6일 모스크바에서 알렉산드르 코즐로프 러시아 극동개발부 장관과 러 북 경제협력위원회 제9차 회의를 열었다 그 전엔 한만혁 북한 노동당 부부장이 모스크바를 찾아 지난 5일 김일성 주석의 첫 소련 공식 방문 및 북러 경제 문화 협정 체결 70주년을 기념하는 사진 전시회 개막식에 참석하고 

defaultdict(float,
            {'지르': 0.8380090388746931,
             '이오': 0.7166481447348724,
             '서울': 0.1563040362101441,
             '여러': 0.4040400723145842,
             '대표적': 0.6517761084190243,
             '제재': 0.7577614840288315,
             '아무리': 0.9643841808393125,
             '나서': 0.3859905171415319,
             '사이': 0.27254113434467125,
             '전면': 0.5255832754472946,
             '트럼프': 0.6818873116574387,
             '가하': 0.46891210863043226,
             '지시': 0.3537214117142472,
             '어야': 0.17545961967100718,
             '도록': 0.34632733755858425,
             '비하': 0.5623795368091453,
             '영향력': 0.8080801446291684,
             '브리': 0.6014574478186875,
             '진적': 0.9643841808393125,
             '통하': 0.22086317350948806,
             '전술': 1.1206882170494568,
             '강화': 0.4954720722088803,
             '미국': 0.30642962955313224,
             '사람': 0.31896057863681343,
             '공식': 0.6252161448405

## KNN search
[KNN -> K 선정하기]
9. K개의 가장 유사한 문서들의 카테고리 추출
10. 카테고리별 투표를 통해 가장 유사한 카테고리 선정

In [442]:
from math import sqrt

candidateList = defaultdict(float)

for token, weight in queryWeight.items():
    for fileName, tfidf in TWM[token].items():
        print('{0} : {1} = {2} * {3}'.format(token, fileName, weight, tfidf))
        candidateList[fileName] += innerProduct(weight, tfidf)
    
for fileName, sumProduct in candidateList.items():
    candidateList[fileName] /= sqrt(DVL[fileName])

지르 : ./0314_DownloadedNewstxts/경제-0000205610.txt = 0.8380090388746931 * 1.5563025007672873
지르 : ./0314_DownloadedNewstxts/경제-0002891740.txt = 0.8380090388746931 * 1.5563025007672873
지르 : ./0314_DownloadedNewstxts/경제-0004189062.txt = 0.8380090388746931 * 1.5563025007672873
지르 : ./0314_DownloadedNewstxts/사회-0002891751.txt = 0.8380090388746931 * 1.5563025007672873
이오 : ./0314_DownloadedNewstxts/IT과학-0002158101.txt = 0.7166481447348724 * 1.380211241711606
이오 : ./0314_DownloadedNewstxts/IT과학-0002158229.txt = 0.7166481447348724 * 1.380211241711606
이오 : ./0314_DownloadedNewstxts/경제-0002515767.txt = 0.7166481447348724 * 1.380211241711606
이오 : ./0314_DownloadedNewstxts/경제-0009127615.txt = 0.7166481447348724 * 1.380211241711606
이오 : ./0314_DownloadedNewstxts/사회-0010696476.txt = 0.7166481447348724 * 1.380211241711606
이오 : ./0314_DownloadedNewstxts/세계-0004109429.txt = 0.7166481447348724 * 1.380211241711606
서울 : ./0314_DownloadedNewstxts/IT과학-0000023756.txt = 0.1563040362101441 * 0.3010299956639812

굴이 : ./0314_DownloadedNewstxts/사회-0001181453.txt = 0.9643841808393125 * 1.8573324964312685
굴이 : ./0314_DownloadedNewstxts/생활문화-0004434766.txt = 0.9643841808393125 * 1.8573324964312685
논평 : ./0314_DownloadedNewstxts/세계-0000372983.txt = 0.9643841808393125 * 1.8573324964312685
논평 : ./0314_DownloadedNewstxts/세계-0004109429.txt = 0.9643841808393125 * 1.8573324964312685
접근 : ./0314_DownloadedNewstxts/IT과학-0002794564.txt = 0.7166481447348724 * 1.380211241711606
접근 : ./0314_DownloadedNewstxts/IT과학-0004108104.txt = 0.7166481447348724 * 1.380211241711606
접근 : ./0314_DownloadedNewstxts/사회-0004335931.txt = 0.7166481447348724 * 1.380211241711606
접근 : ./0314_DownloadedNewstxts/사회-0004430410.txt = 0.7166481447348724 * 1.380211241711606
접근 : ./0314_DownloadedNewstxts/세계-0002447304.txt = 0.7166481447348724 * 1.380211241711606
접근 : ./0314_DownloadedNewstxts/세계-0004109429.txt = 0.7166481447348724 * 1.380211241711606
점진적 : ./0314_DownloadedNewstxts/생활문화-0000749396.txt = 0.9643841808393125 * 1.8573324964312

영향 : ./0314_DownloadedNewstxts/세계-0002446876.txt = 0.4236610558533111 * 0.8159398112730435
영향 : ./0314_DownloadedNewstxts/세계-0003522754.txt = 0.4236610558533111 * 0.8159398112730435
영향 : ./0314_DownloadedNewstxts/세계-0004192327.txt = 0.4236610558533111 * 0.8159398112730435
동안 : ./0314_DownloadedNewstxts/IT과학-0000003610.txt = 0.3537214117142472 * 0.6812412373755872
동안 : ./0314_DownloadedNewstxts/IT과학-0000023756.txt = 0.3537214117142472 * 0.6812412373755872
동안 : ./0314_DownloadedNewstxts/IT과학-0001510642.txt = 0.3537214117142472 * 0.6812412373755872
동안 : ./0314_DownloadedNewstxts/IT과학-0002158095.txt = 0.3537214117142472 * 0.6812412373755872
동안 : ./0314_DownloadedNewstxts/IT과학-0002158101.txt = 0.3537214117142472 * 0.6812412373755872
동안 : ./0314_DownloadedNewstxts/IT과학-0004108104.txt = 0.3537214117142472 * 0.6812412373755872
동안 : ./0314_DownloadedNewstxts/IT과학-0004109529.txt = 0.3537214117142472 * 0.6812412373755872
동안 : ./0314_DownloadedNewstxts/경제-0004109517.txt = 0.3537214117142472 * 0.68

In [443]:
# fileName으로부터 카테고리만 남기기

import re
CategoryList = ['C:/Users/brsta/ICT_AI_AdvanceClass_NLP/0314_DownloadedNewstxts//IT과학-0000003610.txt',
              "C:/Users/brsta/ICT_AI_AdvanceClass_NLP/0314_DownloadedNewstxts//세계-0000372983.txt",
              'C:/Users/brsta/ICT_AI_AdvanceClass_NLP/0314_DownloadedNewstxts//경제-0002221228.txt',
              "C:/Users/brsta/ICT_AI_AdvanceClass_NLP/0314_DownloadedNewstxts//생활문화-0000025147.txt",
             "C:/Users/brsta/ICT_AI_AdvanceClass_NLP/0314_DownloadedNewstxts//사회-0001181453.txt",
              "C:/Users/brsta/ICT_AI_AdvanceClass_NLP/0314_DownloadedNewstxts//정치-0000372172.txt"]

patternList = getPatternList()
CategoryResult = []

def ElimPunc(Filename):
    for _ in ["ElimLongEng", "ElimNumOnly", "Punctuation", "ElimRecWord", "Whitespace"]: 
        Filename = patternList[_].sub("", Filename)
    return Filename

In [444]:
print(ElimPunc(CategoryList[3]))

생활문화


In [445]:
from nltk.tokenize import sent_tokenize

resultList = sorted(candidateList.items(), key=lambda x:x[1], reverse=True)
K = 5

for i, (fileName, similarity) in enumerate(resultList):
    if i < K:
        print('Rank:{0} / Name:{1} / Similarity:{2:.4f}'.format((i+1), fileName, similarity))
        print(getContent(fileName))

Rank:1 / Name:./0314_DownloadedNewstxts/세계-0004109429.txt / Similarity:1.0190





// flash 오류를 우회하기 위한 함수 추가
function _flash_removeCallback() {}

英·獨 이어 인도·UAE까지 5G 장비 배제 거부하자 트럼프, 초강력 대책 만지작[ 김현석 특파원  ] 도널드 트럼프 행정부가 ‘화웨이 봉쇄’를 위해 미국 기업에 대(對)화웨이 수출금지 명령을 내리는 방안을 검토하고 있다. 영국 독일 등 핵심 동맹국들이 5세대(5G) 통신망 구축 때 화웨이 장비를 배제해달라는 미국의 요구를 사실상 거부하자 초강력 대책을 고민하고 있는 것으로 분석된다. 다만 트럼프 대통령이 수출금지 명령을 내리기보다는 미·중 무역협상 카드로 이를 활용할 가능성도 있다.뉴욕타임스(NYT)는 17일(현지시간) “각국의 5G 통신망 구축 사업에서 화웨이를 몰아내려는 미국의 작전이 동맹국의 비협조로 좌초하고 있다”며 “트럼프 행정부가 화웨이에 대한 미국 기업의 수출금지 등 대안을 모색하고 있다”고 보도했다.미국은 동맹국에 ‘화웨이가 백도어(전산망에 몰래 접근할 수 있는 프로그램)가 설치된 통신장비를 통해 기밀을 빼돌릴 수 있다’고 주장해왔다. 마이크 폼페이오 미국 국무장관은 “중국 통신장비를 계속 사용하는 국가와는 정보 교환을 보류하겠다”고 말했다.하지만 영국과 독일, 인도, 아랍에미리트(UAE)는 미국의 요구를 수용하지 않고 있다. 영국은 우려는 있지만 장비와 소프트웨어를 면밀히 조사해 보안 위험을 막을 수 있다고 밝혔다.NYT는 보다폰, 도이치텔레콤 등 유럽 주요 통신사가 이미 화웨이 장비를 많이 채택한 상황이어서 5G 전환 때 화웨이를 배제하면 비용과 시간에서 큰 부담이 생긴다고 보도했다. 또 유럽 국가들에 중국은 미국만큼이나 큰 교역국이어서 명시적 사용금지 조치를 하면 중국의 보복을 부를 것이라는 우려도 있다.이에 따라 트럼프 행정부는 동맹국 협조 없이 화웨이의 세계 5G 통신망 장악을

In [447]:
from collections import Counter

# print(resultList)
FinalIRResult = []
for i in range(len(resultList)):
    FinalIRResult.append(ElimPunc(resultList[i][0]))
    #print(ElimPunc(resultList[i][0]))
# print(FinalIRResult)
modList = Counter(FinalIRResult).most_common()


# 분류하는부분 추가요망

print("-> Given Query : " + X_validation[5][1])
print("\n-> 해당 쿼리의 카테고리는 " + "'" + modList[0][0] + "'" + " 입니다.")

-> Given Query :    베트남 북미 협상 결렬 후 김 위원장 방러 가능성 한층 커져최근 북러 고위인사 접촉 빈번해져 에 러 과의 공조 과시 의도 푸틴 김정은 방러 초청 제작 최자윤 사진합성 일러스트 모스크바 베이징 연합뉴스 유철종 심재훈 특파원 김정은 북한 국무위원장의 최측근 인사인 김창선 북한 국무위원회 부장이 러시아 수도 모스크바를 방문한 것으로 알려졌다 모스크바와 중국 베이징 소식통은 김 위원장의 의전 책임자인 김 부장이 베이징을 거쳐 지난 19일 현지시간 모스크바에 도착했다고 전했다 김 부장은 여전히 모스크바에 머무는 것으로 전해졌다 김 부장은 모스크바에서 러시아 측과 김 위원장의 방러 의전 문제를 논의할 것으로 예상된다 김정은 위원장을 최측근에서 보좌하는 비서실장 격인 김 부장은 김 위원장의 대외 방문 의전 책임자로 그의 모스크바 방문은 김 위원장의 방러가 임박했음을 시사하는 것으로 해석된다 김 부장의 모스크바 방문은 지난달 베트남 북미 협상 결렬 이후 김정은 위원장의 러시아 방문 가능성이 한층 커졌다는 관측이 제기되는 가운데 이루어졌다 북한은 베트남 북미 정상회담에서 자신들이 내세운 요구 조건 수용을 거부한 미국 측을 압박하기 위해 우방인 중국 및 러시아와의 유대 관계를 한층 강화할 것으로 예상돼 왔다 이 같은 맥락에서 김 위원장이 이미 여러 차례 방문한 중국에 이어 조만간 러시아를 찾을 가능성이 커졌다는 분석이 제기됐었다 최근 들어 북러 고위 인사 접촉이 한층 긴밀해진 것도 주목을 받았다 임천일 북한 외무성 부상 차관 이 지난 14일 모스크바를 방문해 이고리 모르굴로프 러시아 외무부 아태지역 담당 차관과 회담했다 김정은 북한 국무위원장 타스 연합뉴스 이에 앞서 김영재 북한 대외경제상도 6일 모스크바에서 알렉산드르 코즐로프 러시아 극동개발부 장관과 러 북 경제협력위원회 제9차 회의를 열었다 그 전엔 한만혁 북한 노동당 부부장이 모스크바를 찾아 지난 5일 김일성 주석의 첫 소련 공식 방문 및 북러 경제 문화 협정 체결 70주년을 기념하는 

## 성능평가하기
- 좋은 성능이 나온 케이스 : 
    - 데이터(문서개수) : 800개, 단어(Lexicon) : 76000개, 모델 : KNN(K=32) <br>
    - 보통 데이터 개수에 squared root를 씌운 값을 K로 두는 게 경험상 좋음. (문서 개수가 900개면 K는 약 30)
- 구현아이디어
    - 전체 뉴스 기사(collections)를 sklearn의 train_test_split로 나눈 후 trainset(X_train)으로 다시 KNN학습 진행
    - X_train과 validationset(X_validation)으로 10-fold CV 수행, 적절한 하이퍼파라미터 탐색